# Deep agents
Пример использования с GigaChat

In [1]:
!pip install deepagents langchain_gigachat python_dotenv -q -U


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
from typing import Literal

from dotenv import find_dotenv, load_dotenv
from langchain_gigachat import GigaChat
from tavily import TavilyClient

from deepagents import create_deep_agent

/Users/knkrestnikov/Documents/giga/workshops/.venv/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/Users/knkrestnikov/Documents/giga/workshops/.venv/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/Users/knkrestnikov/Documents/giga/workshops/.venv/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: typing.NotRequired 

In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [4]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [5]:
# Search tool to use to do research
def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Run a web search"""
    tavily_async_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
    search_docs = tavily_async_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )
    return search_docs

In [26]:
llm = GigaChat(
    verify_ssl_certs=False,
    # model="GigaChat-2-Reasoning",
    model="GigaChat-2-Max",
    profanity_check=False,
    top_p=0,
    timeout=600
)
# llm = llm.bind(additional_fields={"reasoning_effort": "medium"})

In [ ]:
sub_research_prompt = """Только твой ИТОГОВЫЙ ответ будет передан пользователю.  
У него НЕ будет информации ни о чем, кроме твоего итогового сообщения,  
поэтому твой финальный отчет должен быть именно твоим финальным сообщением!"""

In [28]:
research_sub_agent = {
    "name": "research-agent",
    "description": """Используется для исследования более сложных вопросов.  
Давай этому исследователю только одну тему за раз.  
Не передавай этому исследователю несколько подтем одновременно.  
Вместо этого разбей большую тему на необходимые компоненты  
и запусти несколько research-agent параллельно, по одному на каждый подтем.""",
    "prompt": sub_research_prompt,
    "tools": ["internet_search"]
}

In [29]:
sub_critique_prompt = """Ты — внимательный редактор. Твоя задача — критиковать отчет.

Отчет находится в файле `final_report.md`.

Вопрос или тему для этого отчета можно найти в файле `question.txt`.

Пользователь может попросить тебя обратить внимание на конкретные области для критики отчета. Отвечай пользователю подробной критикой отчета. Указывай, что можно улучшить.

Ты можешь использовать инструмент поиска, чтобы находить информацию, если это поможет тебе критиковать отчет.

Не пиши сам(а) в файл `final_report.md`.

Что нужно проверить:
- Проверь, что каждая секция имеет подходящее название.
- Проверь, что отчет написан в стиле эссе или учебника — он должен быть насыщен текстом, а не представлять собой просто список маркеров!
- Проверь, что отчет всеобъемлющий. Если какие-либо абзацы или разделы слишком короткие или в них не хватает важных деталей — укажи на это.
- Проверь, что статья охватывает ключевые области отрасли, обеспечивает общее понимание и не упускает важные части.
- Проверь, что статья глубоко анализирует причины, последствия и тенденции, предоставляя ценные инсайты.
- Проверь, что статья точно следует теме исследования и напрямую отвечает на вопросы.
- Проверь, что статья имеет четкую структуру, грамотный язык и легко читается.
"""

In [30]:
critique_sub_agent = {
    "name": "critique-agent",
    "description": "Используется для критического анализа итогового отчета. Предоставьте этому агенту информацию о том, как вы хотите, чтобы он критиковал отчет.",
    "prompt": sub_critique_prompt,
}

In [31]:
# Prompt prefix to steer the agent to be an expert researcher
research_instructions = """Ты — эксперт-исследователь. Твоя задача — провести глубокое исследование и затем написать выверенный отчет.

Первое, что тебе нужно сделать — записать исходный вопрос пользователя в `question.txt`, чтобы у тебя была его копия.

Используй research-agent для проведения глубокого исследования. Он будет отвечать на твои вопросы/темы подробными ответами.

Когда ты считаешь, что у тебя достаточно информации для написания итогового отчета, запиши его в `final_report.md`.

Ты можешь вызвать critique-agent, чтобы получить критику итогового отчета. После этого (если нужно) ты можешь провести дополнительное исследование и отредактировать `final_report.md`.  
Ты можешь повторять этот процесс столько раз, сколько потребуется, пока не будешь доволен результатом.

Редактируй файл только один раз за раз (если ты вызываешь этот инструмент параллельно, могут возникнуть конфликты).

Ниже приведены инструкции по написанию итогового отчета:

<report_instructions>

ВНИМАНИЕ: Убедись, что ответ написан на том же языке, что и сообщения человека!  
Если ты составляешь план todo — обязательно отметь в плане, на каком языке должен быть отчет, чтобы не забыть!  
Важно: язык отчета должен совпадать с языком ВОПРОСА, а не с языком/страной, о которых идет речь.

Пожалуйста, создай подробный ответ на общее исследовательское задание, который:
1. Хорошо структурирован и имеет правильные заголовки (# для названия, ## для разделов, ### для подразделов)
2. Включает конкретные факты и инсайты из исследования
3. Ссылается на соответствующие источники в формате [Title](URL)
4. Содержит сбалансированный, полный анализ. Будь максимально подробным и включай всю информацию, относящуюся к исследовательскому вопросу. Люди используют тебя для глубоких исследований и ожидают детальные, полные ответы.
5. Содержит раздел «Sources» в конце со всеми использованными ссылками

Ты можешь структурировать отчет по-разному. Вот несколько примеров:

Если нужно сравнить две вещи:
1/ введение  
2/ обзор темы A  
3/ обзор темы B  
4/ сравнение A и B  
5/ заключение  

Если нужно вернуть список:
1/ список или таблица элементов  
или  
1/ элемент 1  
2/ элемент 2  
3/ элемент 3  

Если нужно сделать обзор или общий доклад:
1/ обзор темы  
2/ концепт 1  
3/ концепт 2  
4/ концепт 3  
5/ заключение  

Если вопрос можно ответить одним разделом — сделай это:
1/ ответ  

ПОМНИ: «Раздел» — это гибкое понятие. Структурируй так, как будет лучше для читателя.

Для каждого раздела отчета:
- Используй простой, ясный язык
- Для названий разделов используй ## (Markdown)
- Никогда не упоминай себя как автора
- Не описывай, что ты делаешь — просто пиши текст
- Каждый раздел должен быть достаточно длинным, чтобы глубоко раскрыть тему. Отчет — это глубокое исследование, а не краткая справка.
- Используй списки, когда это уместно, но по умолчанию пиши абзацами.

ПОМНИ:
Задание и исследование могут быть на английском, но итоговый ответ должен быть на правильном языке.  
Оформляй отчет в четком markdown с правильной структурой и ссылками на источники, когда это уместно.

<Citation Rules>
- Каждому уникальному URL присваивается свой номер
- В конце отчета добавляется раздел ### Sources с пронумерованными ссылками
- Нумерация источников должна идти подряд без пропусков
- Каждый источник — отдельная строка списка в markdown
- Пример:
  [1] Название источника: URL  
  [2] Название источника: URL  
- Цитаты крайне важны. Следи за тем, чтобы они были корректными, так как пользователи часто переходят по ним для уточнения информации.
</Citation Rules>
</report_instructions>

У тебя есть доступ к нескольким инструментам.

## `internet_search`

Используй этот инструмент, чтобы выполнить поиск в интернете по заданному запросу. Можно указать количество результатов, тему и включение «сырого» контента.

"""

In [33]:
instructions = """При использовании инструмента Task передавай задания в описание по одному. Помни, что у тебя есть список задач в todo; не передавай агенту все задачи сразу — отправляй их по одной!
При использовании инструмента write_todos не меняй статус задачи на done, пока она фактически не выполнена: сначала передай задачу соответствующему агенту, а помечай как done только когда задача точно уже завершена!"""


In [34]:
# Create the agent
agent = create_deep_agent(
    [internet_search],
    research_instructions + '\n\n' + instructions,
    model=llm,
    subagents=[critique_sub_agent, research_sub_agent],
).with_config({"recursion_limit": 80})


/Users/knkrestnikov/Documents/giga/workshops/.venv/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/Users/knkrestnikov/Documents/giga/workshops/.venv/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/Users/knkrestnikov/Documents/giga/workshops/.venv/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: typing.NotRequired 

In [38]:
inputs={"messages": [{
        "role": "user",
        "content": """Разработай для меня план проведения выходного дня с путешествием из Москвы в Карелию на 3 дня включая выходные.
        Распиши подробно в каких отелях остановиться, какой транспорт выбрать, где обедать и ужинать, какие экскурсии посетить. 
        Оцени затраты при различных сценариях"""
        }]
       }
print_stream(agent.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Разработай для меня план проведения выходного дня с путешествием из Москвы в Карелию на 3 дня включая выходные.
        Распиши подробно в каких отелях остановиться, какой транспорт выбрать, где обедать и ужинать, какие экскурсии посетить. 
        Оцени затраты при различных сценариях
================================== Ai Message ==================================
Tool Calls:
  write_file (5fcb33b9-45cf-4a87-b8b9-72104634aff1)
 Call ID: 5fcb33b9-45cf-4a87-b8b9-72104634aff1
  Args:
    content: Разработай для меня план проведения выходного дня с путешествием из Москвы в Карелию на 3 дня включая выходные.
        Распиши подробно в каких отелях остановиться, какой транспорт выбрать, где обедать и ужинать, какие экскурсии посетить. 
        Оцени затраты при различных сценариях
    file_path: question.txt
================================= Tool Message =================================
Name: write_file

Upda